In [1]:
import pandas as pd

df_model_data_CAN = pd.read_csv('../data/processed/CAN_Categorized_Products_and_Services.csv')
df_model_data_USA = pd.read_csv('../data/processed/US_Categorized_Products_and_Services.csv')
df_Canada_CPI = pd.read_csv('../data/processed/Canada_CPI_Processed_2018_2019.csv')
df_USA_CPI = pd.read_csv('../data/processed/USA_CPI_Processed_2018_2019.csv')

In [2]:
from sklearn.preprocessing import StandardScaler

df_USA_CPI_melt = pd.melt(df_USA_CPI, var_name = 'Products and product groups',value_name = 'VALUE', id_vars = 'REF_DATE')

df_Canada_CPI_Scaled = df_Canada_CPI.copy()
df_Canada_CPI_NonScaled = df_Canada_CPI.copy()

df_USA_CPI_Scaled = df_USA_CPI_melt.copy()
df_USA_CPI_NonScaled = df_USA_CPI_melt.copy()
for unique_product in df_Canada_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_Canada_CPI_Scaled['Products and product groups']==unique_product
    df_Canada_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_Canada_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()
for unique_product in df_USA_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_USA_CPI_Scaled['Products and product groups']==unique_product
    df_USA_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_USA_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()

In [3]:
dict_CAN = df_model_data_CAN.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()
dict_USA = df_model_data_USA.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()

df_Canada_CPI_Scaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_Canada_CPI_NonScaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_USA_CPI_Scaled['Category'] = df_USA_CPI_Scaled['Products and product groups'].map(dict_USA)
df_USA_CPI_NonScaled['Category'] = df_USA_CPI_NonScaled['Products and product groups'].map(dict_USA)


In [4]:
#make groupby dataframe
df_Canada_CPI_Cat = df_Canada_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby = df_Canada_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_Canada_CPI_Cat_NonScaled = df_Canada_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby_NonScaled = df_Canada_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat = df_USA_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby = df_USA_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat_NonScaled = df_USA_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby_NonScaled = df_USA_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

In [5]:
for column in df_Canada_CPI_groupby['Category'].unique():
    print(column)

???
Alcohol & Tobacco
Appliances & Equipment
Clothing & Footwear
Communication
Education & Reading
Energy
Furniture & Home Decor
Gardening
Groceries
Health & Personal Care
Household Supplies
Housing & Utilities
Housing, Rent, Tax  & Insurance
Recreation & Entertainment
Services
Transportation
Utilities
Vehicles


In [6]:
import statsmodels.formula.api as smf
import numpy as np
import altair as alt
from scipy.signal import detrend
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.seasonal import seasonal_decompose, MSTL
def plot_structure(df, category, date1, date2, feature = 'Category', goodtype = None, color = None):
    print("Product: ", category)
    df_in_question = df.copy()
    if goodtype == None:
        df_in_question = df_in_question[df_in_question[feature] == category]
    else:
        df_in_question = df_in_question[(df_in_question[feature] == category) & (df_in_question['GoodType'] == goodtype)]
    df_in_question = df_in_question[(df_in_question['REF_DATE']>=date1) & (df_in_question['REF_DATE']<=date2)]
    df_in_question = df_in_question.sort_values('REF_DATE')
    if color == None:
        chart = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE',
            color=feature
        )
    else:
        chart = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE',
            color=color
        )
    display(chart)
    return None

def plot_supply_and_demand_canada(sales_df, product, date1 = '2018-05-01', date2 = '2018-09-01', principlestats_cat = 'Total inventory, estimated values of total inventory at end of the month', principlestats_cat2 = 'Unfilled orders, estimated values of orders at end of month'):
    df_good = sales_df[(sales_df['GoodType'] == product) & ((sales_df['PrincipleStats'] == principlestats_cat) | (sales_df['PrincipleStats'] == principlestats_cat2))].copy()
    
    scaler = StandardScaler()
    scaler2 = StandardScaler()
    mask1 = (df_good['PrincipleStats']==principlestats_cat)
    mask2 = (df_good['PrincipleStats']==principlestats_cat2)
    print(df_good.loc[mask1, 'VALUE'].head())
    
    scaler.fit(df_good[df_good[mask1]]['VALUE'].values.reshape(-1, 1))
    scaler2.fit(df_good[df_good[mask2]]['VALUE'].values.reshape(-1, 1))
    df_good.loc[mask1, "VALUE"] = scaler.transform(df_good.loc[mask1, 'VALUE'].values.reshape(-1, 1))
    df_good.loc[mask2, "VALUE"] = scaler2.transform(df_good.loc[mask2, 'VALUE'].values.reshape(-1, 1))
    df_good = df_good[(df_good['REF_DATE']>=date1) & (df_good['REF_DATE']<=date2)]
    chart1 = alt.Chart(df_good).mark_point().encode(
    x='REF_DATE',
    y='VALUE',
    color = 'PrincipleStats',
    
    ).properties(title = product).interactive()
    return chart1


def regression_discontinuity_model(df, category, date1, date2, date3, date4 = None, feature = 'Category', goodtype = None, seasonality = None, period = [5, 7]):
    print("Product: ", category)
    df_in_question = df.copy()
    if goodtype ==None: 
        df_in_question = df_in_question[df_in_question[feature] == category]
    else:
        df_in_question = df_in_question[(df_in_question[feature] == category) & (df_in_question['GoodType'] == goodtype)]
    df_in_question = df_in_question.sort_values('REF_DATE')
    if seasonality == None:
        df_in_question['VALUE_DIFF'] = df_in_question['VALUE'].diff()
    
        df_in_question.dropna(subset=['VALUE_DIFF'], inplace=True)
        df_in_question['VALUE_DETREND'] = detrend(df_in_question['VALUE_DIFF'])
        df_in_question = df_in_question[(df_in_question['REF_DATE']>=date1) & (df_in_question['REF_DATE']<=date2)]
        chart = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE_DIFF',
            color=feature
        )
        chart2 = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE',
            color=feature
        )
        chart3 = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE_DETREND',
            color=feature
        )
        display(chart2)
        display(chart)
        display(chart3)
    else:
        df_value = df_in_question['VALUE']
        df_value.index = pd.to_datetime(df_in_question['REF_DATE'])
        
        season_model = MSTL(df_value, periods=period).fit()
        
        df_in_question['VALUE_SEASON'] = season_model.seasonal[season_model.seasonal.columns[0]].tolist()
        df_in_question['RESID'] = season_model.resid.tolist()

        df_in_question['VALUE_DIF'] = df_in_question['VALUE'].values - df_in_question['VALUE_SEASON'].values
        
        df_in_question['TREND'] = season_model.trend.tolist()
        df_in_question['VALUE_DIFF'] = df_in_question['VALUE_DIF'].diff()
        df_in_question.dropna(subset=['VALUE_DIFF'], inplace=True)
        df_in_question['VALUE_DETREND'] = detrend(df_in_question['VALUE_DIFF'])
        df_in_question = df_in_question[(df_in_question['REF_DATE']>=date1) & (df_in_question['REF_DATE']<=date2)]
        chart = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE_DIFF',
            color=feature
        )
        chart2 = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE',
            color=feature
        )
        chart3 = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='VALUE_DETREND',
            color=feature
        )
        chart4 = alt.Chart(df_in_question).mark_line().encode(
            x='REF_DATE',
            y='TREND',
            color=feature
        )
        display(chart2)
        display(chart)
        display(chart3)
        display(chart4)
    
    
    df_in_question['treat'] = np.where((df_in_question['REF_DATE']>=date3) & (df_in_question['REF_DATE']<=date4), 1, 0)
    if date4==None:
        df_in_question['above_or_below'] = np.where(df_in_question['REF_DATE']>date3, 1, 0)
    else:
        df_in_question['above_or_below'] = np.where(df_in_question['REF_DATE']>date4, 1, 0)
    
    df_in_question = df_in_question.sort_values(by=['Category', 'REF_DATE'])
    
    
    dict_replace = {unique_date: count for count, unique_date in enumerate(df_in_question['REF_DATE'].unique())}
    
    df_in_question['Date_Replaced'] = df_in_question['REF_DATE'].map(dict_replace)
    
    for date, num_date in zip(df_in_question['REF_DATE'].iloc[:], df_in_question['Date_Replaced'].iloc[:]):
        if date == date3:
            final_date = num_date
            
    df_in_question['Num_Date'] = df_in_question['Date_Replaced'] - final_date
    
    df_in_question['Num_Date'] = df_in_question['Num_Date'].astype(float)
    if seasonality == None:
        if date4==None:
            model = smf.ols(formula = 'VALUE_DETREND ~ above_or_below + Num_Date + above_or_below:Num_Date', data=df_in_question).fit()
        else:
        #first_stage_model
            
            first_stage_model = smf.ols(formula = 'treat ~ above_or_below + Num_Date', data=df_in_question).fit()
            print(first_stage_model.summary())
            
            df_in_question['PredTreatment'] = first_stage_model.predict(df_in_question)
            
            
            model = smf.ols(formula = 'VALUE_DETREND ~ PredTreatment + Num_Date + PredTreatment:Num_Date', data=df_in_question).fit()
    else:
        if date4==None:
            model = smf.ols(formula = 'TREND ~ above_or_below + Num_Date + above_or_below:Num_Date', data=df_in_question).fit()
        else:
        #first_stage_model
            
            first_stage_model = smf.ols(formula = 'treat ~ above_or_below + Num_Date', data=df_in_question).fit()
            print(first_stage_model.summary())
            
            df_in_question['PredTreatment'] = first_stage_model.predict(df_in_question)
            
            
            model = smf.ols(formula = 'TREND ~ PredTreatment + Num_Date + PredTreatment:Num_Date', data=df_in_question).fit()
        
    return model

def differences_differences(df, category1, category2, date1, date2, date3, date4=None, feature='Category'):
    df_in_question = df.copy()
    
    
    df_differences_differences = df_in_question[(df_in_question[feature]==category1) | (df_in_question[feature]==category2)]
    df_differences_differences = df_differences_differences[(df_differences_differences['REF_DATE']<=date2)&(df_differences_differences['REF_DATE']>=date1)]
    scale = StandardScaler()
        
    df_differences_differences = df_differences_differences[(df_differences_differences[feature]==category1)|(df_differences_differences[feature]==category2)]
    
    if date4==None:
        pass
    else:
        mask = (df_differences_differences['REF_DATE']<=date4)&(df_differences_differences['REF_DATE']>=date3)
        mask2 = df_differences_differences['REF_DATE']==date3
        new_value = df_differences_differences[mask]['VALUE'].mean()
        df_differences_differences.loc[mask2, "VALUE"] = new_value
        df_differences_differences = df_differences_differences[(df_differences_differences['REF_DATE']<=date3) | (df_differences_differences['REF_DATE']>date4)]

    df_differences_differences['VALUE'] = scale.fit_transform(df_differences_differences['VALUE'].values.reshape(-1, 1))
    
    
    df_differences_differences['post'] = np.where(df_differences_differences['REF_DATE']>date3, 1, 0)
    df_differences_differences['tariff_non_tariffed'] = np.where(df_differences_differences['Category']==category1, 1, 0)
    
    model = smf.ols(formula = 'VALUE ~  tariff_non_tariffed + post + tariff_non_tariffed:post', data=df_differences_differences).fit().summary()
    
    
    return model



In [7]:
#test parallel trends assumption:
def plot_for_parallel_trends(df, date1, date2, category_tariff, category_non_tariff, category_3 = None):
    df_Canada_CPI_Scaled_US_on_Canada_Tariffs = df[(df['REF_DATE']<=date2)&(df['REF_DATE']>=date1)]
    scale = StandardScaler()
    if category_3 ==None:
        df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1 = df_Canada_CPI_Scaled_US_on_Canada_Tariffs[(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_non_tariff)]
    else:
        df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1 = df_Canada_CPI_Scaled_US_on_Canada_Tariffs[(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_non_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_3)]
    df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))
    chart1 = alt.Chart(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1).mark_line().encode(
        x='REF_DATE',
        y='VALUE',
        color = 'Category'
    )

    return chart1

In [8]:
plot_for_parallel_trends(df_Canada_CPI_groupby, '2018-06-01', '2018-09-01', 'Groceries', 'Housing, Rent, Tax  & Insurance')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [9]:
print(differences_differences(df_Canada_CPI_Scaled, 'Groceries', 'Housing, Rent, Tax  & Insurance', '2018-06-01', '2018-09-01', '2018-08-01'))
differences_differences(df_Canada_CPI_groupby, 'Groceries', 'Housing, Rent, Tax  & Insurance', '2018-06-01', '2018-09-01', '2018-08-01')

                            OLS Regression Results                            
Dep. Variable:                  VALUE   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     5.296
Date:                Sat, 15 Mar 2025   Prob (F-statistic):            0.00141
Time:                        18:03:03   Log-Likelihood:                -451.89
No. Observations:                 324   AIC:                             911.8
Df Residuals:                     320   BIC:                             926.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     5.452
Date:                Sat, 15 Mar 2025   Prob (F-statistic):             0.0675
Time:                        18:03:03   Log-Likelihood:                -4.8434
No. Observations:                   8   AIC:                             17.69
Df Residuals:                       4   BIC:                             18.00
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.1618      0.362      0.447      0.678      -0.843       1.167
tariff_non_tariffed         -0.4554      0.512     -0.890      0.424      -1.877       0.966
post                         1.7699      0.724      2.445      0.071      -0.240       3.780
tariff_non_tariffed:post    -3.0122      1.024     -2.942      0.042      -5.855      -0.170
==============================================================================
Omnibus:                        0.120   Durbin-Watson:                   1.035
Prob(Omnibus):                  0.942   Jarque-Bera (JB):                0.325
Skew:                           0.007   Prob(JB):                        0.850
Kurtosis:                       2.013   Cond. No.                         6.50
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
#It seems like the groupby shows some interaction while the grouping of all items in grocerys does not
#let us investigate into specific grocery categories to see which groceries may have been impacted by the tariffs

In [11]:

grocery_items = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries')]['Products and product groups'].unique()

grocery_dataframe = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries') | (df_Canada_CPI_NonScaled['Category'] == 'Housing, Rent, Tax  & Insurance')]


dict_grocery = food_categories = {
    'Apples': 'Fruit',
    'Baby foods': 'Prepared Foods',
    'Bakery and cereal products (excluding baby food)': 'Bakery and Cereal',
    'Bakery products': 'Bakery and Cereal',
    'Bananas': 'Fruit',
    'Bread, rolls and buns': 'Bakery and Cereal',
    'Breakfast cereal and other cereal products (excluding baby food)': 'Bakery and Cereal',
    'Butter': 'Dairy',
    'Canned and other preserved fish': 'Fish and Seafood',
    'Canned vegetables and other vegetable preparations': 'Vegetables',
    'Cereal products (excluding baby food)': 'Bakery and Cereal',
    'Cheese': 'Dairy',
    'Coffee': 'Beverages',
    'Coffee and tea': 'Beverages',
    'Condiments, spices and vinegars': 'Condiments and Spices',
    'Confectionery': 'Sweets',
    'Cookies and crackers': 'Bakery and Cereal',
    'Dairy products': 'Dairy',
    'Dairy products and eggs': 'Dairy',
    'Edible fats and oils': 'Fats and Oils',
    'Eggs': 'Dairy',
    'Fish': 'Fish and Seafood',
    'Fish, seafood and other marine products': 'Fish and Seafood',
    'Flour and flour-based mixes': 'Bakery and Cereal',
    'Food': 'General Category',
    'Food and energy': 'General Category',
    'Food purchased from cafeterias and other restaurants': 'Dining Out',
    'Food purchased from fast food and take-out restaurants': 'Dining Out',
    'Food purchased from restaurants': 'Dining Out',
    'Food purchased from stores': 'General Category',
    'Food purchased from table-service restaurants': 'Dining Out',
    'Fresh fruit': 'Fruit',
    'Fresh fruit and vegetables': 'Produce',
    'Fresh milk': 'Dairy',
    'Fresh or frozen beef': 'Meat',
    'Fresh or frozen chicken': 'Meat',
    'Fresh or frozen fish (including portions and fish sticks)': 'Fish and Seafood',
    'Fresh or frozen meat (excluding poultry)': 'Meat',
    'Fresh or frozen pork': 'Meat',
    'Fresh or frozen poultry': 'Meat',
    'Fresh vegetables': 'Vegetables',
    'Frozen and dried vegetables': 'Vegetables',
    'Frozen food preparations': 'Prepared Foods',
    'Fruit juices': 'Beverages',
    'Fruit, fruit preparations and nuts': 'Fruit',
    'Goods excluding food purchased from stores': 'General Category',
    'Goods excluding food purchased from stores and energy': 'General Category',
    'Ham and bacon': 'Meat',
    'Ice cream and related products': 'Dairy',
    'Lettuce': 'Vegetables',
    'Margarine': 'Fats and Oils',
    'Meat': 'Meat',
    'Non-alcoholic beverages': 'Beverages',
    'Non-durable goods excluding food purchased from stores': 'General Category',
    'Non-durable goods excluding food purchased from stores and energy': 'General Category',
    'Nuts and seeds': 'Nuts and Seeds',
    'Oranges': 'Fruit',
    'Pasta products': 'Pasta',
    'Pet food and supplies': 'Pet Supplies',
    'Potatoes': 'Vegetables',
    'Preserved fruit and fruit preparations': 'Fruit',
    'Preserved vegetables and vegetable preparations': 'Vegetables',
    'Processed meat': 'Meat',
    'Rice and rice-based mixes': 'Grains',
    'Seafood and other marine products': 'Fish and Seafood',
    'Soup': 'Prepared Foods',
    'Sugar and confectionery': 'Sweets',
    'Sugar and syrup': 'Sweets',
    'Tea': 'Beverages',
    'Tomatoes': 'Vegetables',
    'Vegetables and vegetable preparations': 'Vegetables',
    'Homeowners': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' replacement cost': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' maintenance and repairs': 'Housing, Rent, Tax  & Insurance',
    'Mortgage interest cost': 'Housing, Rent, Tax  & Insurance',
    'Property taxes': 'Housing, Rent, Tax  & Insurance',
    'Rent': 'Housing, Rent, Tax  & Insurance',
    'Rented accommodation': 'Housing, Rent, Tax  & Insurance',
    'Shelter': 'Housing, Rent, Tax  & Insurance',
    'Shelter (1986)': 'Housing, Rent, Tax  & Insurance',
    'Tenants\' maintenance, repairs and other expenses': 'Housing, Rent, Tax  & Insurance'

}



In [12]:
grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)

grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)

grocery_dataframe_groupby = grocery_dataframe.groupby(['REF_DATE', 'Category']).mean().reset_index()

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/3400313063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/3400313063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)


In [13]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Fish and Seafood', 'Dairy')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [14]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Grains', 'Pasta', 'Vegetables')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [15]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Sweets', 'Fruit')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [16]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Meat', 'Pet Supplies')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [17]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'General Category', 'Dining Out')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_28135/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [18]:
#It seems most items follow a sharp downward trend in august when china tariffs were implemented and were not as affected by Canadian tariffs
#Only dairy and seafood and fish seem to follow the parallel trends assumption 
#let us investigate differences in differences to get an idea of the impact of the tariffs on groceries that followed the aforementioned pattern
#these groceries seem to have to do with everyday food group except carbohydrates and vegetables
differences_differences(grocery_dataframe_groupby, 'Fish and Seafood', 'Dairy', '2018-06-01', '2018-09-01', '2018-08-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     3.131
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.150
Time:                        18:03:03   Log-Likelihood:                -6.5179
No. Observations:                   8   AIC:                             21.04
Df Residuals:                       4   BIC:                             21.35
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.4852      0.446     -1.087      0.338      -1.724       0.754
tariff_non_tariffed          0.9117      0.631      1.445      0.222      -0.840       2.664
post                         2.0470      0.892      2.294      0.084      -0.431       4.525
tariff_non_tariffed:post    -3.8589      1.262     -3.058      0.038      -7.363      -0.355
==============================================================================
Omnibus:                        0.173   Durbin-Watson:                   0.955
Prob(Omnibus):                  0.917   Jarque-Bera (JB):                0.344
Skew:                           0.188   Prob(JB):                        0.842
Kurtosis:                       2.057   Cond. No.                         6.50
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
differences_differences(grocery_dataframe, 'Fish and Seafood', 'Dairy', '2018-06-01', '2018-09-01', '2018-08-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.066
Method:                 Least Squares   F-statistic:                   0.02335
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.995
Time:                        18:03:03   Log-Likelihood:                -68.071
No. Observations:                  48   AIC:                             144.1
Df Residuals:                      44   BIC:                             151.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.0231      0.228     -0.101      0.920      -0.482       0.436
tariff_non_tariffed          0.0430      0.353      0.122      0.904      -0.668       0.754
post                         0.0965      0.455      0.212      0.833      -0.821       1.014
tariff_non_tariffed:post    -0.1819      0.706     -0.258      0.798      -1.604       1.240
==============================================================================
Omnibus:                        0.255   Durbin-Watson:                   0.313
Prob(Omnibus):                  0.881   Jarque-Bera (JB):                0.420
Skew:                          -0.136   Prob(JB):                        0.811
Kurtosis:                       2.631   Cond. No.                         6.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
#It seems the groupby shows a significant difference but when looking at the individual items, the difference is not significant
#perhaps there was not enough data and the set was underpowered but it provides some evidence from the plots that the tariffs may have had a slight impact on goods - let us investigate further

In [21]:
df_sales = pd.read_csv("../data/processed/Canada_Sales_Processed.csv")

In [22]:
df_sales.columns = ['PrincipleStats', 'GoodType', 'REF_DATE', 'VALUE']
df_sales['PrincipleStats'].unique()

array(['Finished goods manufactured, estimated values at end of  month',
       'Goods or work in process, estimated values at end of month',
       'New orders, estimated values of orders received during month',
       'Raw materials, fuel, supplies, components, estimated values at end of month',
       'Sales of goods manufactured (shipments)',
       'Total inventory, estimated values of total inventory at end of the month',
       'Unfilled orders, estimated values of orders at end of month'],
      dtype=object)

In [23]:
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]")

6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


ValueError: Boolean array expected for the condition, not object

In [24]:
#it seems more inventory meant that less orders were filled - this could be due to the tariffs - and it also seems that there were fewer unfilled orders meaning less demand for food manufacturing which means supply went down and inflation should go up.
#but since demand and inventory are similar, it is hard to say if the tariffs had an impact on inflation and the original models without the groupby may be right 

In [25]:
#lets look earlier
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", '2018-01-01', '2018-06-01')

6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [ ]:
model2 = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Groceries', '2017-11-01', '2018-06-01', '2018-02-01', '2018-04-01')

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     1130.
Date:                Sat, 15 Mar 2025   Prob (F-statistic):          3.48e-198
Time:                        17:59:47   Log-Likelihood:                 63.160
No. Observations:                 568   AIC:                            -120.3
Df Residuals:                     565   BIC:                            -107.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6250      0.011     57.

In [27]:

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                  0.009755
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.999
Time:                        17:59:47   Log-Likelihood:                -774.11
No. Observations:                 568   AIC:                             1556.
Df Residuals:                     564   BIC:                             1574.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0010      0.054     -0.018      0.986      -0.107       0.105
PredTreatment             -0.0090      0.133     -0.068      0.946      -0.270       0.252
Num_Date                   0.0002      0.018      0.009      0.993      -0.036       0.036
PredTreatment:Num_Date     0.0109      0.072      0.151      0.880      -0.130       0.152
==============================================================================
Omnibus:                        5.551   Durbin-Watson:                   2.943
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                6.805
Skew:                           0.095   Prob(JB):                       0.0333
Kurtosis:                       3.502   Cond. No.                         8.83
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
#it seems like american tariffs had little immediate impact on Canadian groceries as mentioned supply was still high - it is hard to disentangle the impact of the tariffs and the news so perhaps we should look for a more long term approach to gain more insight to what happened with groceries


plot_for_parallel_trends(df_Canada_CPI_groupby, '2017-01-01', '2020-02-01', 'Groceries', 'Education & Reading')



/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_27971/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [29]:
#it is hard to quantify the impact of groceries based on specific tariffs due to the serial correlation of tariffs but looking from the start of the breakdown in China talks to the Chinese hike tax we will look at RD between these periods and see if we can draw conclusions
#there does seem to be greater variance from the start of the Intellectual Property Trials to the Chinese hike in tariffs that started propagating like a wave with each tariff with minimal activity following the results of the IP investigation - let us investigate further
model_groceries_longterm = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2017-01-01', '2018-04-01', '2017-08-01')

model_groceries_longterm.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     3.043
Date:                Sat, 15 Mar 2025   Prob (F-statistic):             0.0744
Time:                        17:59:47   Log-Likelihood:                 7.0097
No. Observations:                  15   AIC:                            -6.019
Df Residuals:                      11   BIC:                            -3.187
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.0225      0.121      0.187      0.855      -0.243       0.288
above_or_below             -0.3867      0.183     -2.110      0.059      -0.790       0.017
Num_Date                   -0.0218      0.033     -0.651      0.528      -0.095       0.052
above_or_below:Num_Date     0.0883      0.043      2.045      0.066      -0.007       0.183
==============================================================================
Omnibus:                        1.916   Durbin-Watson:                   2.464
Prob(Omnibus):                  0.384   Jarque-Bera (JB):                1.189
Skew:                          -0.407   Prob(JB):                        0.552
Kurtosis:                       1.887   Cond. No.                         25.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
#Next lets compare with RD the tariff / IP trails period to the Chinese hike in tariffs
model_groceries_longterm_recovery = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Groceries', '2018-02-01', '2020-02-01', '2018-09-01', '2019-02-01')

model_groceries_longterm_recovery.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     1270.
Date:                Sat, 15 Mar 2025   Prob (F-statistic):               0.00
Time:                        17:59:47   Log-Likelihood:                -219.21
No. Observations:                1775   AIC:                             444.4
Df Residuals:                    1772   BIC:                             460.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5262      0.009     57.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                  0.001528
Date:                Sat, 15 Mar 2025   Prob (F-statistic):               1.00
Time:                        17:59:47   Log-Likelihood:                -2774.7
No. Observations:                1775   AIC:                             5557.
Df Residuals:                    1771   BIC:                             5579.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.0013      0.064      0.021      0.983      -0.125       0.128
PredTreatment             -0.0067      0.170     -0.039      0.969      -0.339       0.326
Num_Date                1.155e-05      0.007      0.002      0.999      -0.014       0.014
PredTreatment:Num_Date     0.0004      0.028      0.015      0.988      -0.055       0.056
==============================================================================
Omnibus:                        3.883   Durbin-Watson:                   2.949
Prob(Omnibus):                  0.143   Jarque-Bera (JB):                4.055
Skew:                          -0.063   Prob(JB):                        0.132
Kurtosis:                       3.198   Cond. No.                         58.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
#The IP trials overall impact including the US tariffs on Canada shows a reduction on inflation in groceries in Canada while the Chinese hike in tariffs to the end of the data does not show a significant impact on groceries with the groupby

#the smaller impact of each tariff appear to cause a wave that ripples through time causing greater variance in the data, each individual month of tariffs increased inflation at the treatment period but appeared to reduce inflation the following month before another wave occurs. 

#overall groceries were not as impacted when grouped together as there were some groceries influenced such as seafood while other groceries such as dairy remained unimpacted 


#In other words the tariffs had a short term impact on inflation but contributed to larger ripples in waves that caused greater variance in the data.

#The news of breakdown in China talks momentarily caused economic shock that influenced the agricultural industry to decrease food supply in Canada and increase inflation
# and the proceeding recovery after the break down in China talks and beginning of IP trade investigations, did appear to make grocery inflation lower for a short period of time.





In [32]:

#let us plot the char for sales for food manufucturing during this time period again for further investigate invesigation

plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", "2017-02-01", "2018-12-01")



6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [33]:
#It appears that demand for food manufacturing was lower than supply during the trade talk breakdown which means supply for food was limited during the trade talks which caused inflation but demand rose following the trade talk breakdown and remained unaffected throughout the tariffs which is why groceries were largely unaffected



In [34]:
#while inventory did increase meaning a decrease in supply, there was still enough demand and that is why groceries were not impacted despite the sharp decrease in August 2018 perhaps mainly due to the China tariffs on US goods.



In [35]:
#let us look at what happened in American groceries due to the tariffs and compare the effect of the tariffs on groceries for US tariffs vs Canadian tariffs
#they appear to have a similar effect on American goods

plot_structure(df_USA_CPI_groupby, 'Groceries', '2017-01-01', '2020-02-01')



Product:  Groceries


alt.Chart(...)

In [ ]:
#It appears that the American grocery dataset follows the Canadian with waves except the breakdown in china talks led to a decrease in inflation that was equilibriated by the IP investigation
# We will look at the IP investigation later but it appears trumps tariffs on Canadian goods had only a small impact on American groceries but the combination of Canadian announcement of tariffs and canadian tariffs and chinese tariffs on American exported goods may have had a larger impact, let us investigate further
model_USA_RC_grocery_cutoff = regression_discontinuity_model(df_USA_CPI_groupby, 'Groceries', '2017-10-01', '2018-06-01', '2018-03-01', '2018-04-01')
model_USA_RC_grocery_cutoff = regression_discontinuity_model(df_USA_CPI_groupby, 'Groceries', '2018-01-01', '2018-10-01', '2018-05-01', '2018-08-01')

model_USA_RC_grocery_cutoff.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     5.464
Date:                Sat, 15 Mar 2025   Prob (F-statistic):             0.0445
Time:                        17:59:47   Log-Likelihood:               -0.20393
No. Observations:                   9   AIC:                             6.408
Df Residuals:                       6   BIC:                             7.000
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6366      0.161      3.

...

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     13.50
Date:                Sat, 15 Mar 2025   Prob (F-statistic):            0.00396
Time:                        17:59:47   Log-Likelihood:                0.84845
No. Observations:                  10   AIC:                             4.303
Df Residuals:                       7   BIC:                             5.211
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5941      0.096      6.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                 -0.164
Method:                 Least Squares   F-statistic:                    0.5777
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.651
Time:                        17:59:47   Log-Likelihood:                 12.664
No. Observations:                  10   AIC:                            -17.33
Df Residuals:                       6   BIC:                            -16.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0521      0.039     -1.353      0.225      -0.146       0.042
PredTreatment              0.0291      0.096      0.304      0.772      -0.205       0.264
Num_Date                  -0.0064      0.010     -0.619      0.559      -0.032       0.019
PredTreatment:Num_Date     0.0242      0.036      0.677      0.524      -0.063       0.112
==============================================================================
Omnibus:                        0.558   Durbin-Watson:                   2.956
Prob(Omnibus):                  0.757   Jarque-Bera (JB):                0.495
Skew:                          -0.420   Prob(JB):                        0.781
Kurtosis:                       2.305   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
#It appear neither american tariffs nor canadian/chinese tariffs had much of an impact on American groceries but did cause more variance meaning some groceries were more impacted then others - let us investigate further



In [ ]:
print(df_USA_CPI_groupby['Category'].unique())
plot_for_parallel_trends(df_USA_CPI_groupby, '2017-06-01', '2019-12-01', 'Groceries', 'Education & Reading', category_3 = None)

['Alcohol & Tobacco' 'Appliances & Equipment' 'Clothing & Footwear'
 'Communication' 'Education & Reading' 'Energy' 'Furniture & Home Decor'
 'Groceries' 'Health & Personal Care' 'Household Supplies'
 'Housing & Utilities' 'Housing, Rent, Tax & Insurance'
 'Recreation & Entertainment' 'Services' 'Transportation' 'Uncategorized'
 'Utilities' 'Vehicles']


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_27971/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [ ]:
differences_differences(df_USA_CPI_Scaled, 'Groceries', 'Education & Reading', '2018-03-01', '2018-05-01', '2018-04-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.042
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.376
Time:                        17:59:47   Log-Likelihood:                -194.22
No. Observations:                 138   AIC:                             396.4
Df Residuals:                     134   BIC:                             408.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.4096      0.268     -1.528      0.129      -0.940       0.121
tariff_non_tariffed          0.4745      0.291      1.630      0.106      -0.101       1.050
post                         0.1885      0.464      0.406      0.685      -0.730       1.107
tariff_non_tariffed:post    -0.1965      0.504     -0.390      0.697      -1.194       0.801
==============================================================================
Omnibus:                       22.452   Durbin-Watson:                   0.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.892
Skew:                           0.907   Prob(JB):                     3.23e-07
Kurtosis:                       4.382   Cond. No.                         11.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
differences_differences(df_USA_CPI_groupby, 'Groceries', 'Education & Reading', '2018-03-01', '2018-05-01', '2018-04-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     4.176
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.199
Time:                        17:59:47   Log-Likelihood:                -2.5649
No. Observations:                   6   AIC:                             13.13
Df Residuals:                       2   BIC:                             12.30
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.1668      0.454     -2.568      0.124      -3.122       0.788
tariff_non_tariffed          2.2703      0.643      3.533      0.072      -0.495       5.035
post                         1.2923      0.787      1.642      0.242      -2.094       4.679
tariff_non_tariffed:post    -2.3945      1.113     -2.151      0.164      -7.184       2.395
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.353
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.459
Skew:                          -0.000   Prob(JB):                        0.795
Kurtosis:                       1.646   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:


#differences in differences appears to confirm little impact on overall groceries - let us examine why with datasets of export and import





In [42]:
USA_sales = pd.read_csv("../data/processed/USA_Sales_Processed.csv")
USA_sales_groceries_vehicles = pd.read_csv("../data/processed/USA_Sales_Groceries_Vehicles_Processed.csv")
print(USA_sales.tail())
print(USA_sales_groceries_vehicles.tail())

                    cat_idx                                    dt_idx  \
575623  Other Durable Goods         New Orders Percent Change Monthly   
575624  Other Durable Goods    Unfilled Orders Percent Change Monthly   
575625  Other Durable Goods    Unfilled Orders Percent Change Monthly   
575626  Other Durable Goods  Total Inventories Percent Change Monthly   
575627  Other Durable Goods  Total Inventories Percent Change Monthly   

        geo_idx  is_adj  val    REF_DATE  
575623        1       1 -0.5  2023-01-01  
575624        1       0  0.4  2023-01-01  
575625        1       1 -0.9  2023-01-01  
575626        1       0  2.0  2023-01-01  
575627        1       1  0.3  2023-01-01  
                                      cat_idx dt_idx  et_idx  geo_idx  is_adj  \
185555  72251X: Limited Service Eating Places    NaN       5        1       1   
185556  72251X: Limited Service Eating Places    NaN       5        1       1   
185557  72251X: Limited Service Eating Places    NaN       5

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_27971/834534902.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  USA_sales_groceries_vehicles = pd.read_csv("../data/processed/USA_Sales_Groceries_Vehicles_Processed.csv")


In [43]:
for column in USA_sales_groceries_vehicles['cat_idx'].unique():
    print(column)

44X72: Retail Trade and Food Services
44Y72: Retail Trade and Food Services, ex Auto
44Z72: Retail Trade and Food Services, ex Gas
44W72: Retail Trade and Food Services, ex Auto and Gas
44000: Retail Trade
4400A: Retail Trade, ex Auto
4400C: GAFO
441: Motor Vehicle and Parts Dealers
4411,4412: Auto and Other Motor Vehicles
4411: Automobile Dealers
44111: New Car Dealers
44112: Used Car Dealers
4413: Automotive Parts, Acc., and Tire Stores
4423X: Furniture, Home Furn, Electronics, and Appliance Stores
442: Furniture and Home Furnishings Stores
4421: Furniture Stores
4422: Home Furnishings Stores
44221: Floor Covering Stores
443: Electronics and Appliance Stores
443141: Household Appliance Stores
444: Building Mat. and Garden Equip. and Supplies Dealers
4441: Building Mat. and Supplies Dealers
44413: Hardware Stores
445: Food and Beverage Stores
4451: Grocery Stores
4453: Beer, Wine, and Liquor Stores
446: Health and Personal Care Stores
44611: Pharmacies and Drug Stores
447: Gasoline St

In [44]:
USA_sales['PrincipleStats'] = USA_sales['dt_idx']
USA_sales['GoodType'] = USA_sales['cat_idx']
USA_sales['VALUE'] = USA_sales["val"]



In [ ]:
for column in USA_sales['PrincipleStats'].unique():
    print(column)

print(sep = '\n')
for column in USA_sales['GoodType'].unique():
    print(column)
print(sep = '\n')
for column in USA_sales.columns:
    print(column)

USA_sales['PrincipleStats'] = USA_sales["PrincipleStats"].astype(str)
USA_sales['GoodType'] = USA_sales['GoodType'].astype(str)

Value of Shipments
New Orders
Unfilled Orders
Total Inventories
Materials and Supplies Inventories
Work in Process Inventories
Finished Goods Inventories
Inventories to Shipments Ratios
Unfilled Orders to Shipments Ratios
Value of Shipments Percent Change Monthly
New Orders Percent Change Monthly
Unfilled Orders Percent Change Monthly
Total Inventories Percent Change Monthly
Materials & Supplies Inventories Percent Change Monthly
Work in Progress Inventories Percent Change Monthly
Finished Goods Inventories Percent Change Monthly

Total Manufacturing
Manufacturing Excluding Transportation
Manufacturing Excluding Defense
Manufacturing with Unfilled Orders
Durable Goods
Wood Products
Nonmetallic Mineral Products
Primary Metals
Iron and Steel Mills and Ferroalloy and Steel Product Manufacturing
Aluminum and Nonferrous Metal Products
Ferrous Metal Foundries
Fabricated Metal Products
Machinery
Farm Machinery and Equipment Manufacturing
Construction Machinery Manufacturing
Mining, Oil, and G

In [46]:
print(USA_sales[USA_sales['PrincipleStats'] == 'Unfilled Orders'])
USA_sales['VALUE'] = USA_sales['VALUE'].astype(float)
plot_supply_and_demand_canada(USA_sales, product = "Food Products", date1 = "2017-01-01", date2 = "2020-02-01", principlestats_cat="Unfilled Orders", principlestats_cat2="Total Inventories")


                                cat_idx           dt_idx  geo_idx  is_adj  \
46                  Total Manufacturing  Unfilled Orders        1       0   
47                  Total Manufacturing  Unfilled Orders        1       0   
48                  Total Manufacturing  Unfilled Orders        1       0   
49                  Total Manufacturing  Unfilled Orders        1       0   
50                  Total Manufacturing  Unfilled Orders        1       0   
...                                 ...              ...      ...     ...   
575585  Durable Goods Excluding Defense  Unfilled Orders        1       1   
575600          Communication Equipment  Unfilled Orders        1       0   
575601          Communication Equipment  Unfilled Orders        1       1   
575616              Other Durable Goods  Unfilled Orders        1       0   
575617              Other Durable Goods  Unfilled Orders        1       1   

              val    REF_DATE   PrincipleStats  \
46       477445.0  1992-0

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

In [48]:

#It appears that imports and exports of food, feeds, and beverages were largely unaffected by the tariffs as exports are always less than imports meaning tariffs did not affect supply of imports and exports seem to be only affected during the chinese tariff hike period meaning the affect of Chinese and Canadian tariffs were not affected until later the demand from other countries

In [49]:
#lets compare the change in variance of groceries in America to Canada to compare the effect of tariffs on groceries in both countries

mask_USA_groceries = df_USA_CPI_Scaled['Category'] == 'Groceries'
mask_Canada_groceries = df_Canada_CPI_Scaled['Category'] == 'Groceries'

df_USA_CPI_Scaled_groceries = df_USA_CPI_Scaled[mask_USA_groceries]
df_Canada_CPI_Scaled_groceries = df_Canada_CPI_Scaled[mask_Canada_groceries]

df_USA_CPI_Scaled_groceries['Country'] = ['USA']*len(df_USA_CPI_Scaled_groceries)
df_Canada_CPI_Scaled_groceries['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_groceries)

df_USA_CPI_Scaled_groceries.drop(columns = ['Products and product groups'], inplace=True)
df_Canada_CPI_Scaled_groceries.drop(columns = ['Products and product groups'], inplace=True)

df_USA_CPI_Scaled_groc_var = df_USA_CPI_Scaled_groceries.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()
df_Canada_CPI_Scaled_groc_var = df_Canada_CPI_Scaled_groceries.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()

df_USA_Canada_groceries = pd.concat([df_USA_CPI_Scaled_groc_var, df_Canada_CPI_Scaled_groc_var])

plot_structure(df_USA_Canada_groceries, 'Groceries', '2017-01-01', '2020-02-01', color = 'Country')


Product:  Groceries


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_26875/4185968530.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_USA_CPI_Scaled_groceries['Country'] = ['USA']*len(df_USA_CPI_Scaled_groceries)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_26875/4185968530.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_groceries['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_groceries)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_26875/4185968530.py:12:

alt.Chart(...)

In [50]:
#It seems like the break down of the China talks decreased variance and the proceeding IP investigation increased variance in America first while Canada had a lag in their decrease.
#The IP investigation and the tariffs appeared to increase variance following the breakdown in talks meaning there was recooperation but America did not quite recover following the end of the tariffs in May 2019. 

In [51]:
plot_structure(df_Canada_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2020-02-01')

Product:  Clothing & Footwear


alt.Chart(...)

In [52]:
#It doesnt appear Canadian clothing and footwear was that impacted by tariffs but let us look closer

In [53]:
plot_supply_and_demand_canada(df_sales, "Leather and allied product manufacturing [316]", "2017-01-01", "2020-02-01")

alt.Chart(...)

In [ ]:

plot_supply_and_demand_canada(df_sales, "Apparel manufacturing [315]", "2017-01-01", "2020-02-01")

alt.Chart(...)

In [55]:
#IT seems like manufacturing demand for clothing took a hit during July which means there were less orders from clothing stores and supply for clothing was decreased

#Inflation should have increased a bit for clothing and footwear but it was the impact of trumps tariffs that caused the reduction in supply - this increase in inflation is difficult to measure in July due to the seasonal nature of clothing and footwear but perhaps we can look at regression discontinuity 

In [56]:
model_clothing_US_tariff = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Clothing & Footwear', '2018-03-01', '2018-09-01', '2018-07-01', '2018-08-01')

model_clothing_US_tariff.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     131.3
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           9.36e-30
Time:                        17:34:29   Log-Likelihood:                -1.2430
No. Observations:                 112   AIC:                             8.486
Df Residuals:                     109   BIC:                             16.64
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6762      0.034     20.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                   0.02316
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.995
Time:                        17:34:29   Log-Likelihood:                -149.79
No. Observations:                 112   AIC:                             307.6
Df Residuals:                     108   BIC:                             318.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0167      0.149     -0.112      0.911      -0.313       0.279
PredTreatment              0.0221      0.284      0.078      0.938      -0.540       0.584
Num_Date                  -0.0019      0.054     -0.036      0.971      -0.109       0.105
PredTreatment:Num_Date     0.0412      0.168      0.245      0.807      -0.292       0.374
==============================================================================
Omnibus:                        2.528   Durbin-Watson:                   2.934
Prob(Omnibus):                  0.283   Jarque-Bera (JB):                2.024
Skew:                           0.197   Prob(JB):                        0.364
Kurtosis:                       3.528   Cond. No.                         7.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
model_clothing_US_tariff_groupby = regression_discontinuity_model(df_Canada_CPI_groupby, 'Clothing & Footwear', '2017-08-01', '2020-01-01', '2019-03-01', '2019-08-01', seasonality=True)

model_clothing_US_tariff_groupby.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     17.66
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           1.25e-05
Time:                        17:34:29   Log-Likelihood:                -2.5346
No. Observations:                  30   AIC:                             11.07
Df Residuals:                      27   BIC:                             15.27
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5446      0.077      7.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  TREND   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     759.1
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           2.01e-25
Time:                        17:34:29   Log-Likelihood:                 53.552
No. Observations:                  30   AIC:                            -99.10
Df Residuals:                      26   BIC:                            -93.50
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.4285      0.013     33.565      0.000       0.402       0.455
PredTreatment             -0.2356      0.031     -7.499      0.000      -0.300      -0.171
Num_Date                   0.0486      0.001     44.020      0.000       0.046       0.051
PredTreatment:Num_Date     0.0257      0.004      6.007      0.000       0.017       0.035
==============================================================================
Omnibus:                        0.823   Durbin-Watson:                   0.438
Prob(Omnibus):                  0.663   Jarque-Bera (JB):                0.858
Skew:                          -0.334   Prob(JB):                        0.651
Kurtosis:                       2.510   Cond. No.                         40.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
#this visual trend is not picked up by statistics - which means there was lower impact on inflation perhaps because of the already abundance of clothing in stores and the already lack of demand

#In fact the effect of the tariffs was not seen until during May 2019 - August 2019 which coincides with food manufacturing increases meaning there was a lack of supply for clothing which is why there was inflation, this impact was significant. 

In [59]:
#It appears to have caused a decrease in orders or manufacturing demand which means less supply for clothing and footwear, however, demand was still above supply in this time frame - the impact may not have been felt until in May 2019 because of the already high supply of clothing and footwear in retail stores



In [60]:
#let us look at the American dataset to see if the same trend is seen

plot_structure(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2020-02-01')

Product:  Clothing & Footwear


alt.Chart(...)

In [61]:
#It seems that both trumps tariffs caused American goods to increase in inflation while Canadian/Chinese tariffs caused a decrease in inflation for American clothings and footwear

#let us investigate further with regression discontinuity

model_clothing_US_tariff = regression_discontinuity_model(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-10-01', '2018-06-01', '2018-02-01', '2018-04-01')

model_clothing_US_tariff.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     10.30
Date:                Sat, 15 Mar 2025   Prob (F-statistic):             0.0115
Time:                        17:34:29   Log-Likelihood:                0.69908
No. Observations:                   9   AIC:                             4.602
Df Residuals:                       6   BIC:                             5.194
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6391      0.116      5.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.015
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.459
Time:                        17:34:29   Log-Likelihood:                 5.2059
No. Observations:                   9   AIC:                            -2.412
Df Residuals:                       5   BIC:                            -1.623
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0232      0.079     -0.292      0.782      -0.227       0.181
PredTreatment              0.2397      0.180      1.332      0.240      -0.223       0.702
Num_Date                  -0.0040      0.025     -0.160      0.879      -0.068       0.060
PredTreatment:Num_Date     0.0198      0.093      0.213      0.839      -0.219       0.258
==============================================================================
Omnibus:                        1.761   Durbin-Watson:                   2.815
Prob(Omnibus):                  0.415   Jarque-Bera (JB):                1.088
Skew:                          -0.615   Prob(JB):                        0.580
Kurtosis:                       1.821   Cond. No.                         8.22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
model_clothing_US_tariff = regression_discontinuity_model(df_USA_CPI_Scaled, 'Clothing & Footwear', '2017-12-01', '2018-04-01', '2018-02-01', '2018-03-01')

model_clothing_US_tariff.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     205.0
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           1.25e-32
Time:                        17:34:29   Log-Likelihood:                 16.192
No. Observations:                  85   AIC:                            -26.38
Df Residuals:                      82   BIC:                            -19.06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.7000      0.027     25.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.037
Method:                 Least Squares   F-statistic:                   0.01105
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.998
Time:                        17:34:29   Log-Likelihood:                -101.66
No. Observations:                  85   AIC:                             211.3
Df Residuals:                      81   BIC:                             221.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0140      0.123     -0.113      0.910      -0.259       0.231
PredTreatment              0.0418      0.267      0.157      0.876      -0.490       0.573
Num_Date                  -0.0078      0.067     -0.117      0.907      -0.141       0.125
PredTreatment:Num_Date    -0.0212      0.249     -0.085      0.932      -0.516       0.473
==============================================================================
Omnibus:                        9.180   Durbin-Watson:                   2.863
Prob(Omnibus):                  0.010   Jarque-Bera (JB):               11.375
Skew:                          -0.513   Prob(JB):                      0.00339
Kurtosis:                       4.469   Cond. No.                         5.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
#let us now look at transportation and see if the tariffs had an impact on transportation

In [ ]:
model_transportation_Canada_tariff = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Transportation', '2018-05-01', '2018-09-01', '2018-07-01')


Product:  Transportation


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [65]:
model_transportation_Canada_tariff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                   0.04281
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.988
Time:                        17:34:30   Log-Likelihood:                -112.90
No. Observations:                  90   AIC:                             233.8
Df Residuals:                      86   BIC:                             243.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.0650      0.187     -0.348      0.729      -0.436       0.306
above_or_below              0.0930      0.494      0.188      0.851      -0.889       1.075
Num_Date                   -0.0482      0.145     -0.333      0.740      -0.336       0.239
above_or_below:Num_Date     0.0334      0.323      0.103      0.918      -0.609       0.676
==============================================================================
Omnibus:                        0.691   Durbin-Watson:                   3.078
Prob(Omnibus):                  0.708   Jarque-Bera (JB):                0.761
Skew:                          -0.036   Prob(JB):                        0.683
Kurtosis:                       2.555   Cond. No.                         10.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
#Initially it seems Canadian tariffs decreased inflation for transportation but let us look further

In [67]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-02-01', '2018-11-01', 'Transportation', 'Energy')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_26875/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [68]:
#it appears that trumps tariffs in March/April had a significant influence on energy which may have caused transportation to rise in May
#trudeau's tariffs in July may have caused a decrease in transportation inflation
#let us see if we can do a differences in differences with housing, rent, tax and insurance which were largely unaffected by the tariffs

plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-01-01', '2018-11-01', 'Transportation', 'Housing, Rent, Tax  & Insurance')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_26875/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [69]:
differences_differences(df_Canada_CPI_groupby_NonScaled, 'Transportation', 'Housing, Rent, Tax  & Insurance', '2018-03-01', '2018-07-01', '2018-05-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     68.60
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           4.92e-05
Time:                        17:34:30   Log-Likelihood:                 3.6297
No. Observations:                  10   AIC:                            0.7405
Df Residuals:                       6   BIC:                             1.951
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.9995      0.125     -7.967      0.000      -1.307      -0.693
tariff_non_tariffed          1.5107      0.177      8.515      0.000       1.077       1.945
post                         0.2041      0.198      1.029      0.343      -0.281       0.689
tariff_non_tariffed:post     0.8129      0.281      2.898      0.027       0.126       1.499
==============================================================================
Omnibus:                        5.853   Durbin-Watson:                   2.316
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                2.092
Skew:                           1.039   Prob(JB):                        0.351
Kurtosis:                       3.836   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
differences_differences(df_Canada_CPI_NonScaled, 'Transportation', 'Housing, Rent, Tax  & Insurance', '2018-03-01', '2018-07-01', '2018-05-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3729
Date:                Sat, 15 Mar 2025   Prob (F-statistic):              0.773
Time:                        17:34:30   Log-Likelihood:                -198.08
No. Observations:                 140   AIC:                             404.2
Df Residuals:                     136   BIC:                             415.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.0841      0.184     -0.456      0.649      -0.449       0.281
tariff_non_tariffed          0.0504      0.230      0.219      0.827      -0.405       0.505
post                         0.0308      0.292      0.106      0.916      -0.546       0.608
tariff_non_tariffed:post     0.1530      0.364      0.421      0.675      -0.566       0.872
==============================================================================
Omnibus:                       34.102   Durbin-Watson:                   0.315
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               70.962
Skew:                           1.044   Prob(JB):                     3.90e-16
Kurtosis:                       5.794   Cond. No.                         7.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
#Despite the visualized increase in transportation costs in May, the differences in differences model shows no significant difference in inflation between transportation and housing, rent, tax and insurance

#This may be due to a violation of the parallel trends assumption since Canadian taxes were implemented in July affected the post period. There seems no good comparator for transportation so we must rely on visualizations to see the impact of the tariffs on transportation
#let us plot how equipment manufacturing was affected by the tariffs

plot_supply_and_demand_canada(df_sales, "Transportation equipment manufacturing [336]", "2018-01-01", "2018-11-01")

alt.Chart(...)

In [ ]:
#It seems trumps tariffs in March/April did not change Total Inventory production until May which may have been a composition of factors such as the global climate 
# where demand for transportation equipment was higher and supply was low is where the inflation occurred
#This means that consumer demand increased or stayed the same for transportation equipment and supply was low which caused inflation to rise in May

#we can quantify this effect using regression discontinuity

model_sales_RC_transportation = regression_discontinuity_model(df_sales, 'Total inventory, estimated values of total inventory at end of the month',
       '2018-01-01', '2018-09-01', '2018-07-01', feature = 'PrincipleStats', goodtype = 'Transportation equipment manufacturing [336]')

Product:  Total inventory, estimated values of total inventory at end of the month


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

KeyError: 'Category'

In [67]:
model_sales_RC_transportation.summary()

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     15.30
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0117
Time:                        17:48:49   Log-Likelihood:                -107.02
No. Observations:                   8   AIC:                             222.0
Df Residuals:                       4   BIC:                             222.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           1.602e+07   2.05e+05     77.949      0.000    1.54e+07    1.66e+07
Num_Date            3.422e+05   5.28e+04      6.487      0.003    1.96e+05    4.89e+05
treatment          -7.985e+05   3.02e+05     -2.648      0.057   -1.64e+06    3.87e+04
Num_Date:treatment -4.455e+05   3.17e+05     -1.408      0.232   -1.32e+06    4.33e+05
==============================================================================
Omnibus:                        0.387   Durbin-Watson:                   1.788
Prob(Omnibus):                  0.824   Jarque-Bera (JB):                0.197
Skew:                          -0.287   Prob(JB):                        0.906
Kurtosis:                       2.488   Cond. No.                         16.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""